In [ ]:
import sys
from pathlib import Path

# Trouver la racine du projet en remontant jusqu'à un dossier qui contient "src"
cwd = Path().resolve()

project_root = None
for p in [cwd] + list(cwd.parents):
    if (p / "src").exists():
        project_root = p
        break

if project_root is None:
    raise RuntimeError(f"Impossible de trouver la racine du projet depuis: {cwd}")

sys.path.insert(0, str(project_root))
project_root


# 01 - Exploration du dataset

Dataset: **Chest X-ray (pneumonia)**
Objectif: Analyser la structure, la distribution et la qualité des données avant tout entrainement.

Questions auxquelles ce notebook répond :
1. Combien d'images par split (train/val/test) et par classe (NORMAL/PNEUMONIA) ?
2. Le dataset est-il équilibré ?
3. À quoi ressemble les images (qualité, contraste, exemples) ?
4. Quelles contraintes techniques en découle pour le processing (tailles, canaux, valeurs, pixels) ?

In [ ]:
from __future__ import annotations

from collections import Counter
from pathlib import Path
import random

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

from src.config import CFG
from src.data.load import index_chest_xray

In [ ]:
random.seed(CFG.SEED)
np.random.seed(CFG.SEED)

## 1) Chargement du dataset

On indexe le dataset : pour chaque split, on récupère la liste des chemins d’images et leurs labels numériques (0 = normal, 1 = pneumonia).


In [ ]:
splits = index_chest_xray()

train_ds = splits["train"]
val_ds = splits["val"]
test_ds = splits["test"]

train_ds.class_to_idx

## 2) Volumes par split

On commence par mesurer combien d’images sont disponibles dans chaque split.

In [ ]:
print("Train:", len(train_ds))
print("Val  :", len(val_ds))
print("Test :", len(test_ds))

## 3) Distribution des classes

On calcule combien d’images appartiennent à chaque classe, pour chaque split.
Cela permet d’identifier un éventuel **déséquilibre** du dataset.

In [ ]:
def class_distribution(ds) -> dict[str, int]:
    counts = Counter(ds.labels)
    idx_to_class = {v: k for k, v in ds.class_to_idx.items()}
    return {idx_to_class[idx]: counts.get(idx, 0) for idx in idx_to_class}

In [ ]:
print("Train:", class_distribution(train_ds))
print("Val  :", class_distribution(val_ds))
print("Test :", class_distribution(test_ds))

In [ ]:
def plot_distribution(ds, title: str) -> None:
    dist = class_distribution(ds)
    classes = list(dist.keys())
    values = list(dist.values())

    plt.figure()
    plt.title(title)
    plt.bar(classes, values)
    plt.xlabel("Classe")
    plt.ylabel("Nombre d'images")
    plt.show()

plot_distribution(train_ds, "Distribution des classes — TRAIN")
plot_distribution(test_ds, "Distribution des classes — TEST")
plot_distribution(val_ds, "Distribution des classes — VAL")

## 4) Inspection visuelle

Avant preprocessing, on affiche quelques exemples d’images pour chaque classe (NORMAL vs PNEUMONIA) afin d’observer :
- la variabilité (contraste, luminosité),
- le bruit,
- la qualité générale.

In [ ]:
def show_images(paths: list[Path], title: str, n: int = 6) -> None:
    n = min(n, len(paths))
    chosen = random.sample(paths, n)

    plt.figure(figsize=(12, 6))
    plt.suptitle(title)

    for i, p in enumerate(chosen, start=1):
        img = Image.open(p)
        plt.subplot(2, (n + 1) // 2, i)
        plt.imshow(img, cmap="gray")
        plt.title(p.name[:20])
        plt.axis("off")

    plt.show()

In [ ]:
normal_label = train_ds.class_to_idx["normal"]
pneumonia_label = train_ds.class_to_idx["pneumonia"]

train_normal_paths = [p for p, y in zip(train_ds.paths, train_ds.labels) if y == normal_label]
train_pneumonia_paths = [p for p, y in zip(train_ds.paths, train_ds.labels) if y == pneumonia_label]

len(train_normal_paths), len(train_pneumonia_paths)

In [ ]:
show_images(train_normal_paths, "Exemples TRAIN — NORMAL", n=6)
show_images(train_pneumonia_paths, "Exemples TRAIN — PNEUMONIA", n=6)

## 5) Vérifications techniques

On inspecte :
- la taille des images (largeur/hauteur),
- le mode (RGB ou grayscale),
- les valeurs de pixels.

Objectif : justifier le preprocessing (resize + normalisation).

In [ ]:
def inspect_sample(ds, n: int = 10) -> None:
    n = min(n, len(ds.paths))
    chosen = random.sample(ds.paths, n)

    sizes = []
    modes = []

    for p in chosen:
        img = Image.open(p)
        sizes.append(img.size)   # (width, height)
        modes.append(img.mode)   # 'RGB', 'L', ...

    print("Exemples de tailles (width, height):", sizes[:5])
    print("Modes rencontrés:", sorted(set(modes)))

inspect_sample(train_ds, n=20)

In [ ]:
def inspect_pixel_range(ds, n: int = 10) -> None:
    n = min(n, len(ds.paths))
    chosen = random.sample(ds.paths, n)

    mins, maxs = [], []
    for p in chosen:
        img = Image.open(p)
        arr = np.array(img)
        mins.append(arr.min())
        maxs.append(arr.max())

    print("Pixel min (échantillon):", min(mins))
    print("Pixel max (échantillon):", max(maxs))

inspect_pixel_range(train_ds, n=20)

## 6) Conclusions 

Observations principales :
- Le dataset est réparti en splits train/val/test et en classes NORMAL/PNEUMONIA.
- Le split **val** est très petit → on l’utilisera surtout pour valider le pipeline, et on pourra créer un split de validation depuis train si besoin.
- Le dataset est **déséquilibré** (plus de PNEUMONIA que de NORMAL) → il faudra interpréter les métriques avec prudence (accuracy seule insuffisante).
- Les images présentent une variabilité (taille, contraste) → preprocessing nécessaire :
  - resize vers une taille fixe (ex: 224x224),
  - normalisation des pixels (0..1),
  - conversion de format cohérente (grayscale ou RGB selon choix).

Prochaine étape (notebook 02) :
- implémenter le preprocessing minimal,
- entraîner un modèle baseline simple (ex: régression logistique) sur train,
- évaluer sur test avec accuracy + precision/recall/F1 + matrice de confusion.      